<a href="https://colab.research.google.com/github/dani1974/DanielaRosasPUC/blob/main/mvp_classifica_o_danielarosas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *** MVP - Sprint II - Aluna : Daniela Rosas ***
## 1º modelos de machine learning para um problema de classificação

Este conjunto de dados fornece uma compilação abrangente de sintomas e perfis de pacientes para uma ampla gama de doenças.

Doença
Febre (Sim/Não)
Tosse (Sim/Não)
Fadiga (Sim/Não)
Dificuldade para Respirar (Sim/Não)
Idade (em anos)
Sexo (Masculino/Feminino)
Pressão Arterial (Normal/Alta)
Nível de Colesterol (Normal/Alto) )
Variável de Resultado (Positivo/Negativo)

In [ ]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes # para importar o dataset diabetes
from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste (holdout)
from sklearn.model_selection import KFold # para preparar os folds da validação cruzada
from sklearn.model_selection import cross_val_score # para executar a validação cruzada
from sklearn.metrics import mean_squared_error # métrica de avaliação MSE
from sklearn.linear_model import LinearRegression # algoritmo Regressão Linear
from sklearn.linear_model import Ridge # algoritmo Regularização Ridge
from sklearn.linear_model import Lasso # algoritmo Regularização Lasso
from sklearn.neighbors import KNeighborsRegressor # algoritmo KNN
from sklearn.tree import DecisionTreeRegressor # algoritmo Árvore de Regressão
from sklearn.svm import SVR # algoritmo SVM
# Importando as bibliotecas necessárias
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt


#####**Carga dos dados**

In [ ]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/dani1974/DanielaRosasPUC/main/Dados/Disease_symptom_and_patient_profile_dataset.csv"

# Lê o arquivo
dataset = pd.read_csv(url, delimiter=',')

# Mostra as primeiras linhas do dataset
dataset.head()

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [ ]:
dataset = dataset.rename(columns={'Disease': 'Doenças','Fever': 'Febre','Cough': 'Tosse','Fatigue': 'Fadiga','Difficulty Breathing': 'Dif_respirar','Age': 'Idade',
                        'Gender': 'Sexo','Blood Pressure': 'Pressão','Cholesterol Level': 'Colesterol','Outcome Variable': 'Target'})

dataset.drop('Doenças',axis=1,inplace=True)
dataset.head()


,Febre,Tosse,Fadiga,Dif_respirar,Idade,Sexo,Pressão,Colesterol,Target
0,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


#####**Preparação dos dados - Separação em bases de treino e teste (holdout)**


In [ ]:
# Separação em bases de treino e teste (holdout)
array = dataset.values
X = array[:,0:8] # atributos
y = array[:,8] # classe (target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7) # faz a divisão

# Criando os folds para a validação cruzada
num_particoes = 10 # número de folds da validação cruzada
kfold = KFold(n_splits=num_particoes, shuffle=True, random_state=7) # faz o particionamento em 10 folds

#####**Preparação dos dados - Criando os folds para a validação cruzada**

In [ ]:
# Criando o pipeline com os modelos
pipelines = []
numeric_features = slice(0, 8)
categorical_features = [7]  # Índice da coluna categórica
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', Pipeline([
            ('ordinal', OrdinalEncoder()),
            ('onehot', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])
pipelines.append(('KNN', Pipeline([('Preprocessor', preprocessor), ('KNN', KNeighborsClassifier())])))
pipelines.append(('CART', Pipeline([('Preprocessor', preprocessor), ('CART', DecisionTreeClassifier())])))
pipelines.append(('NB', Pipeline([('Preprocessor', preprocessor), ('NB', GaussianNB())])))
pipelines.append(('SVM', Pipeline([('Preprocessor', preprocessor), ('SVM', SVC())])))



In [ ]:
# Listas para armazenar os resultados e os nomes dos modelos
results = []
names = []

# Avaliando um modelo por vez
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())  # média e desvio padrão dos 10 resultados da validação cruzada
    print(msg)



ValueError: ignored